# Descriptors Computation
This notebook handles the descriptors computation. 
This procedure involves the following steps:
- For each variable, we estimate its Markov Blanket (MB) by selecting its lagged versions from one time step before and one after. 
- We standardize the time series to avoid varsortability
- Using the estimated MB, we compute a set of descriptors for all possible causal pairs (i.e., $ t-\tau \rightarrow t, \forall \tau$). that characterize the causal relationship between the variable pairs. These descriptors include conditional mutual information terms and other statistical properties that provide insights into the dependencies and interactions between the variables.
- For families of descriptors, we compute the quantiles of their empirical distributions. This step captures the distributional characteristics and aids in feature representation for the classifier.
- The computed descriptors and their quantiles are compiled into an input feature vector. This vector encapsulates the essential characteristics of the causal relationships and serves as the input for the classifier.
- For training data, each input vector is labeled as causal (1) or noncausal (0) based on the original selection criteria from the synthetic data's Directed Acyclic Graph (DAG). This labeling is crucial for supervised learning and model training.
- The labeled dataset, comprising the feature vectors, is used to train a classifier. The classifier learns to predict the likelihood of causal relationships based on the descriptors.
- For unseen time series data, the trained classifier predicts the probability of causal links for each pair of variables. The predictions are based on the computed descriptors for the test data.

Here are the descriptors 
\begin{align}
n  \\
m \\
m/n \\
b: \mathbf{z}_j = b \cdot (\mathbf{z}_i \oplus \mathbf{MB}_j) \text{, where} \oplus\text{indicates vector concatenation}  \\
b: \mathbf{z}_i = b \cdot (\mathbf{z}_j \oplus \mathbf{MB}_i) \text{, where} \oplus\text{indicates vector concatenation}   \\
\operatorname{kurt}(\mathbf{z}_i) = \mathbb{E}[(\mathbf{z}_i-\mathbb{E}[\mathbf{z}_i])^4] / \left(\mathbb{E}[(\mathbf{z}_i-\mathbb{E}[\mathbf{z}_i])^2]\right)^2 - 3 \\ 
\operatorname{kurt}(\mathbf{z}_j) = \mathbb{E}[(\mathbf{z}_j-\mathbb{E}[\mathbf{z}_j])^4] / \left(\mathbb{E}[(\mathbf{z}_j-\mathbb{E}[\mathbf{z}_j])^2]\right)^2 - 3 \\ 
\operatorname{skew}(\mathbf{z}_i) = \mathbb{E}[(\mathbf{z}_i-\mathbb{E}[\mathbf{z}_i])^3] / \left(\mathbb{E}[(\mathbf{z}_i-\mathbb{E}[\mathbf{z}_i])^2]\right)^{3/2} \\
\operatorname{skew}(\mathbf{z}_j) = \mathbb{E}[(\mathbf{z}_j-\mathbb{E}[\mathbf{z}_j])^3] / \left(\mathbb{E}[(\mathbf{z}_j-\mathbb{E}[\mathbf{z}_j])^2]\right)^{3/2}
 \\
\operatorname{HOC_{1,2}}(\mathbf{z}_i, \mathbf{z}_j) = \mathbb{E}\left[ (\mathbf{z}_i - \mathbb{E}[\mathbf{z}_i])^1 \cdot (\mathbf{z}_j - \mathbb{E}[\mathbf{z}_j])^2 \right] \\
\operatorname{HOC_{2,1}}(\mathbf{z}_i, \mathbf{z}_j) = \mathbb{E}\left[ (\mathbf{z}_i - \mathbb{E}[\mathbf{z}_i])^2 \cdot (\mathbf{z}_j - \mathbb{E}[\mathbf{z}_j])^1 \right] \\
\operatorname{HOC_{1,3}}(\mathbf{z}_i, \mathbf{z}_j) = \mathbb{E}\left[ (\mathbf{z}_i - \mathbb{E}[\mathbf{z}_i])^1 \cdot (\mathbf{z}_j - \mathbb{E}[\mathbf{z}_j])^3 \right] \\
\operatorname{HOC_{3,1}}(\mathbf{z}_i, \mathbf{z}_j) = \mathbb{E}\left[ (\mathbf{z}_i - \mathbb{E}[\mathbf{z}_i])^3 \cdot (\mathbf{z}_j - \mathbb{E}[\mathbf{z}_j])^1 \right] \\
    I(\mathbf{z}_i ; \mathbf{z}_j)  \\
    I(\mathbf{z}_j ; \mathbf{z}_i)  \\ 
    I(\mathbf{m}_j^{(k)} ; \mathbf{z}_i) \forall \mathbf{m}_j^{(k)} \in \mathbf{MB}_j  \\
    I(\mathbf{m}_i^{(k)} ; \mathbf{z}_j) \forall \mathbf{m}_i^{(k)} \in \mathbf{MB}_i \\
    I(\mathbf{z}_i ; \mathbf{z}_j | \mathbf{MB}_i \cap \mathbf{MB}_j)  \\
    I(\mathbf{z}_j ; \mathbf{z}_i | \mathbf{MB}_j)  \\
    I(\mathbf{z}_i ; \mathbf{z}_j | \mathbf{MB}_i)  \\
    I(\mathbf{z}_j ; \mathbf{z}_i | \mathbf{MB}_i \cup \mathbf{m}_j^{(k)}) \forall \mathbf{m}_j^{(k)} \in \mathbf{MB}_j  \\
    I(\mathbf{z}_i ; \mathbf{z}_j | \mathbf{MB}_j \cup \mathbf{m}_i^{(k)}) \forall \mathbf{m}_i^{(k)} \in \mathbf{MB}_i \\
    I(\mathbf{z}_i ; \mathbf{m}_j^{(k)} | \mathbf{z}_j) \forall \mathbf{m}_j^{(k)} \in \mathbf{MB}_j  \\
I(\mathbf{z}_j ; \mathbf{m}_i^{(k)} | \mathbf{z}_i) \forall \mathbf{m}_i^{(k)} \in \mathbf{MB}_i  \\
    I(\mathbf{m}_i^{(k_i)} ; \mathbf{m}_j^{(k_j)} | \mathbf{z}_i) \forall (\mathbf{m}_i^{(k_i)}, \mathbf{m}_j^{(k_j)}) \in \mathbf{MB_{i}} \times \mathbf{MB_{j}}  \\
    I(\mathbf{m}_j^{(k_j)} ; \mathbf{m}_i^{(k_i)} | \mathbf{z}_j) \forall (\mathbf{m}_i^{(k_i)}, \mathbf{m}_j^{(k_j)}) \in \mathbf{MB_{i}} \times \mathbf{MB_{j}} \\
        I(\mathbf{m}_i^{(k_{i}^{1})} ; \mathbf{m}_i^{(k_{i}^{2})} | \mathbf{z}_i) - I(\mathbf{m}_i^{(k_{i}^{1})} ; \mathbf{m}_i^{(k_{i}^{2})}) \forall (\mathbf{m}_i^{(k_{i}^{1})}, \mathbf{m}_i^{(k_{i}^{2})}) \in \mathbf{MB_{i}} \times \mathbf{MB_{i}}  \\
    I(\mathbf{m}_j^{(k_{j}^{1})} ; \mathbf{m}_j^{(k_{j}^{2})} | \mathbf{z}_j) - I(\mathbf{m}_j^{(k_{j}^{1})} ; \mathbf{m}_j^{(k_{j}^{2})}) \forall (\mathbf{m}_j^{(k_{j}^{1})}, \mathbf{m}_j^{(k_{j}^{2})}) \in \mathbf{MB_{j}} \times \mathbf{MB_{j}}     
\end{align}

In [1]:
from d2c.descriptors import D2C, DataLoader

In [2]:
N_VARS = 5
MAXLAGS = 3
N_JOBS = 40

The DataLoader class for handling time series data and directed acyclic graphs (DAGs) to prepare for the following stage: the descriptors computation.
The preparation includes:
1. Creating lagged time series: This step involves generating lagged versions of the original time series data. Lagged time series help in capturing temporal dependencies and interactions between variables at different time steps.
2. Flattening the original dictionaries into coherent lists: The original data, which may be stored in nested dictionaries, is flattened into lists. This transformation ensures that the data is in a consistent and accessible format for further processing.
3. Renaming the nodes of the DAGs: The nodes of the Directed Acyclic Graphs (DAGs) are renamed to maintain consistency and clarity. This step is crucial for accurately representing the causal relationships between variables in the DAGs.


In [3]:
dataloader = DataLoader(n_variables = N_VARS,
                    maxlags = MAXLAGS)
dataloader.from_pickle('example/synthetic_data_train.pkl')

original_observations = dataloader.get_original_observations()
lagged_flattened_observations = dataloader.get_observations()
flattened_dags = dataloader.get_dags()

We are now ready for the core of our methodology: the D2C method. <br>
This method starts from a list of observations and dags and computes the corresponding descritpors, storing them in a dataframe. <br>
The D2C class gets the following arguments: 
- `observations` (list): List of observations (pd.DataFrame) corresponding to each DAG.
- `dags` (list): List of directed acyclic graphs (DAGs) representing causal relationships.
- `couples_to_consider_per_dag` (int, optional): To speedup, one can consider only a limited number of possible couples of variables. Couples are chosen to respect a specific ratio of causal/noncausal. Therefore, a DAG must be available: it can only be used for training data. For testing purposes only, we recommend using `D2CWrapper` instead. Defaults to -1 (all couples).  
- `n_variables` (int, optional): Number of variables in the time series. Defaults to 3.
- `maxlags` (int, optional): Maximum number of lags in the time series. Defaults to 3.
- `seed` (int, optional): Random seed for reproducibility. Defaults to 42.
- `n_jobs` (int, optional): Number of parallel jobs to run. Defaults to 1.
 - `full` (bool, optional): computes the whole set of descriptors rather than just a subset. Defaults to True. 



In [4]:
d2c = D2C(observations=lagged_flattened_observations,
        dags=flattened_dags, 
        couples_to_consider_per_dag=20, 
        n_variables=N_VARS, 
        maxlags=MAXLAGS,
        seed=42,
        n_jobs=N_JOBS,
        full=True)

d2c.initialize()

Now let's look at the descriptors. 

We remind our variable naming convention: <br>
- A time series of `n_variables` dimensions will have names from 0 to `n_variables - 1` to refer to the variables at time `t` (present)
- names from `n_variables` to `n_variables*2 - 1` will indicate the same variable at time `t-1` (1-lag)
- names from `n_variables*2` to `n_variables*3 - 1` will indicate the same variable at time `t-2` (2-lag)  <br>
For example if `n_variables = 5`, the line where `edge_source` is 12 and `edge_dest` is 4, refers to the link between variable `3` at `t-2` to variable `5` at time `t`

In [5]:
descriptors_df_train = d2c.get_descriptors_df()
print(descriptors_df_train.columns)
descriptors_df_train

Index(['graph_id', 'edge_source', 'edge_dest', 'is_causal', 'coeff_cause',
       'coeff_eff', 'HOC_3_1', 'HOC_1_2', 'HOC_2_1', 'HOC_1_3', 'kurtosis_ca',
       'kurtosis_ef', 'mca_mef_cau_q0', 'mca_mef_cau_q1', 'mca_mef_cau_q2',
       'mca_mef_eff_q0', 'mca_mef_eff_q1', 'mca_mef_eff_q2', 'cau_m_eff_q0',
       'cau_m_eff_q1', 'cau_m_eff_q2', 'eff_m_cau_q0', 'eff_m_cau_q1',
       'eff_m_cau_q2', 'm_cau_q0', 'm_cau_q1', 'm_cau_q2', 'com_cau',
       'cau_eff', 'eff_cau', 'eff_cau_mbeff', 'cau_eff_mbcau',
       'eff_cau_mbcau_plus_q0', 'eff_cau_mbcau_plus_q1',
       'eff_cau_mbcau_plus_q2', 'cau_eff_mbeff_plus_q0',
       'cau_eff_mbeff_plus_q1', 'cau_eff_mbeff_plus_q2', 'm_eff_q0',
       'm_eff_q1', 'm_eff_q2', 'mca_mca_cau_q0', 'mca_mca_cau_q1',
       'mca_mca_cau_q2', 'mbe_mbe_eff_q0', 'mbe_mbe_eff_q1', 'mbe_mbe_eff_q2',
       'n_samples', 'n_features', 'n_features/n_samples', 'skewness_ca',
       'skewness_ef'],
      dtype='object')


,graph_id,edge_source,edge_dest,is_causal,coeff_cause,coeff_eff,HOC_3_1,HOC_1_2,HOC_2_1,HOC_1_3,...,mca_mca_cau_q1,mca_mca_cau_q2,mbe_mbe_eff_q0,mbe_mbe_eff_q1,mbe_mbe_eff_q2,n_samples,n_features,n_features/n_samples,skewness_ca,skewness_ef
0,0,11,1,1,1.244976e-02,0.013721,0.051125,0.082896,0.064003,-0.222003,...,0.0,0.013471,0.0,0.0,0.010264,247,20,0.080972,-0.105222,-0.119371
1,0,1,11,0,1.372115e-02,0.012450,-0.222003,0.064003,0.082896,0.051125,...,0.0,0.010264,0.0,0.0,0.013471,247,20,0.080972,-0.119371,-0.105222
2,0,14,4,1,-1.982190e-03,0.011765,0.567162,-0.057519,-0.132904,0.421689,...,0.0,0.014072,0.0,0.0,0.011671,247,20,0.080972,-0.071466,-0.071288
3,0,4,14,0,1.176531e-02,-0.001982,0.421689,-0.132904,-0.057519,0.567162,...,0.0,0.011671,0.0,0.0,0.014072,247,20,0.080972,-0.071288,-0.071466
4,0,7,0,1,3.941213e-02,0.082011,0.543090,-0.013511,-0.104255,0.363200,...,0.0,0.005817,0.0,0.0,0.000000,247,20,0.080972,-0.021983,0.153496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1433,79,9,0,0,-4.912845e-02,-0.046653,-0.369035,0.143122,0.126069,-0.685040,...,0.0,0.000000,0.0,0.0,0.000000,247,20,0.080972,0.084436,-0.166015
1434,79,5,1,0,-1.820083e-02,-0.001662,0.094858,-0.152048,-0.123522,-0.171476,...,0.0,0.000000,0.0,0.0,0.000000,247,20,0.080972,-0.152121,0.037282
1435,79,9,1,0,8.615237e-02,0.064781,0.114678,0.085267,0.043925,0.305056,...,0.0,0.000000,0.0,0.0,0.000000,247,20,0.080972,0.084436,0.037282
1436,79,13,0,0,2.829680e+13,0.164443,0.627862,-0.156683,-0.099588,0.917458,...,0.0,0.000000,0.0,0.0,0.000000,247,20,0.080972,-0.042320,-0.166015


Now we can see if we can learn something from these descriptors. We will perform a 50% train/test split and run a classifier. 

In [6]:
dataloader = DataLoader(n_variables = N_VARS,
                    maxlags = MAXLAGS)
dataloader.from_pickle('example/synthetic_data_test.pkl')

d2c = D2C(observations=dataloader.get_observations(),
        dags=dataloader.get_dags(), 
        couples_to_consider_per_dag=20, 
        n_variables=N_VARS, 
        maxlags=MAXLAGS,
        seed=42,
        n_jobs=N_JOBS,
        full=True)

d2c.initialize()

descriptors_df_test = d2c.get_descriptors_df()

In [7]:
X_train = descriptors_df_train.drop(columns=['graph_id','edge_source','edge_dest','is_causal'])
y_train = descriptors_df_train['is_causal']
X_test = descriptors_df_test.drop(columns=['graph_id','edge_source','edge_dest','is_causal'])
y_test = descriptors_df_test['is_causal']

from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix

clf = BalancedRandomForestClassifier(n_estimators=10, max_depth=None, random_state=0, sampling_strategy='auto',replacement=True,bootstrap=True)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
y_pred_proba = clf.predict_proba(X_test)

print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"ROC AUC Score: {roc_auc_score(y_test, y_pred_proba[:, 1]):.4f}")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.7927
ROC AUC Score: 0.8869
Confusion Matrix:
[[425  52]
 [ 96 141]]


Scores are promising. We now save the descriptors.

In [9]:
# save the descriptors_df to disk
import pickle
filename = 'example/descriptors_df_train.pkl'
pickle.dump(descriptors_df_train, open(filename, 'wb'))